In [58]:
# Installations
!pip install yfinance
!pip install ta
!pip install PyPortfolioOpt

In [59]:
# Imports
import yfinance as yf
import pandas as pd
import numpy as np
from pypfopt.expected_returns import mean_historical_return
from pypfopt.risk_models import CovarianceShrinkage
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import objective_functions
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from scipy.stats import norm

In [60]:
# Data Initialisation
tickers = ["D05.SI", "Z74.SI", "BN4.SI", "C38U.SI", "V03.SI", "F34.SI", "S68.SI"]

start_date = "2015-01-01"
end_date = "2015-12-31"

data = yf.download(tickers, start = start_date, end = end_date)

price_data = data['Adj Close']

price_data.head()

[*********************100%***********************]  7 of 7 completed


Ticker,BN4.SI,C38U.SI,D05.SI,F34.SI,S68.SI,V03.SI,Z74.SI
Date,,,,,,,
2015-01-02 00:00:00+00:00,3.109825,1.223353,12.056107,2.374518,5.324235,4.940667,2.573631
2015-01-05 00:00:00+00:00,3.050156,1.229350,11.785576,2.345472,5.296721,4.921976,2.567015
2015-01-06 00:00:00+00:00,2.920287,1.211360,11.662075,2.316426,5.241690,4.890824,2.540550
2015-01-07 00:00:00+00:00,2.930816,1.247340,11.644436,2.301903,5.303600,4.872134,2.553782
2015-01-08 00:00:00+00:00,2.972937,1.217356,11.809100,2.352733,5.379268,4.921976,2.606711


## Base Portfolio

In [61]:
mu = mean_historical_return(price_data)
S = CovarianceShrinkage(price_data).ledoit_wolf()
ef = EfficientFrontier(mu, S)
ef.add_objective(objective_functions.L2_reg, gamma=0.1)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()
print(cleaned_weights)

OrderedDict([('BN4.SI', 0.0), ('C38U.SI', 0.0), ('D05.SI', 0.0), ('F34.SI', 0.0), ('S68.SI', 0.09455), ('V03.SI', 0.90545), ('Z74.SI', 0.0)])


/usr/local/lib/python3.10/dist-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(


In [62]:
ef.portfolio_performance(verbose=True)

Expected annual return: 9.0%
Annual volatility: 14.0%
Sharpe Ratio: 0.50


(0.08985143812863924, 0.1397231681051613, 0.4999273855289784)

In [63]:
# Backtesting (Option 1)
w = np.array(list(cleaned_weights.values())) # Convert odict_values to a NumPy array
if w.sum() != 1:
  w = w / w.sum()

start_date = "2023-01-01"
end_date = "2023-12-31"

newData = yf.download(tickers, start=start_date, end=end_date)['Adj Close']

daily_returns = newData.pct_change().dropna()

portfolio_returns = daily_returns.dot(w)
total_return = (1 + portfolio_returns).prod() - 1
portfolio_std_dev = portfolio_returns.std() * np.sqrt(252)
risk_free_rate = 0.03

sharpe_ratio = (portfolio_returns.mean() * 252 - risk_free_rate) / portfolio_std_dev

print("Total Portfolio Return for 2023:", total_return)
print("Annualized Portfolio Standard Deviation:", portfolio_std_dev)
print("Sharpe Ratio:", sharpe_ratio)

[*********************100%***********************]  7 of 7 completed

Total Portfolio Return for 2023: -0.1350014452387478
Annualized Portfolio Standard Deviation: 0.1824700434811735
Sharpe Ratio: -0.8808511499669683


In [64]:
# Backtesting (Option 2)
latest_prices = newData.iloc[0]
da = DiscreteAllocation(cleaned_weights, latest_prices, total_portfolio_value=10000000)
allocation, leftover = da.lp_portfolio()
print(allocation)

{'S68.SI': 115227, 'V03.SI': 587870}


In [65]:
allocation_series = pd.Series(allocation)

portfolio_value = (newData * allocation_series).sum(axis=1)

daily_returns = portfolio_value.pct_change().dropna()

total_return = (portfolio_value.iloc[-1] / portfolio_value.iloc[0]) - 1

std_dev = daily_returns.std()

risk_free_rate = 0
sharpe_ratio = (daily_returns.mean() - risk_free_rate) / std_dev * np.sqrt(252)
print(f"Total Portfolio Return: {total_return:.2%}")
print(f"Portfolio Standard Deviation (daily): {std_dev:.4f}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")

Total Portfolio Return: -13.22%
Portfolio Standard Deviation (daily): 0.0113
Sharpe Ratio: -0.7141


## Variant 1 Portfolio : Growth-Oriented Allocation

In [67]:
sector_mapper = {
    "D05.SI": "Financials",   # DBS Bank
    "Z74.SI": "Telecommunications",  # Singtel
    "BN4.SI": "Industrials",  # Keppel Corp
    "C38U.SI": "Real Estate", # CapitaLand Integrated Commercial Trust
    "V03.SI": "Consumer Staples",  # Venture Corporation
    "F34.SI": "Consumer Discretionary",  # Wilmar International
    "S68.SI": "Financials"    # Singapore Exchange
}

sector_lower = {"Financials": 0.3}  # At least 30% in Financials

sector_upper = {
    "Financials": 0.5,  # Max 50% in Financials
    "Industrials": 0.3,  # Max 30% in Industrials
    "Telecommunications": 0.15,  # Max 15% in Telecommunications
    "Real Estate": 0.15  # Max 15% in Real Estate
}

ef = EfficientFrontier(mu, S)
ef.add_objective(objective_functions.L2_reg, gamma=0.1)
ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

print("Optimized Weights with Variant 1:", cleaned_weights)

ef.portfolio_performance(verbose=True)

w = np.array(list(cleaned_weights.values())) # Convert odict_values to a NumPy array
if w.sum() != 1:
  w = w / w.sum()

daily_returns = newData.pct_change().dropna()
portfolio_returns = daily_returns.dot(w)

total_return = (1 + portfolio_returns).prod() - 1
portfolio_std_dev = portfolio_returns.std() * np.sqrt(252)
risk_free_rate = 0.03

# Download STI data (used as market benchmark)
sti_data = yf.download('^STI', start=start_date, end=end_date)['Adj Close']
sti_returns = sti_data.pct_change().dropna()

# Ensure both portfolio_returns and sti_returns have the same length and are aligned
aligned_returns = pd.concat([portfolio_returns, sti_returns], axis=1).dropna()
portfolio_returns_aligned = aligned_returns.iloc[:, 0]
sti_returns_aligned = aligned_returns.iloc[:, 1]

# Beta calculation: covariance of portfolio with market (STI), divided by variance of market (STI)
cov_matrix = np.cov(portfolio_returns_aligned, sti_returns_aligned)
beta = cov_matrix[0, 1] / cov_matrix[1, 1]

# Annualized Alpha: alpha = (portfolio return - risk-free rate) - beta * (market return - risk-free rate)
portfolio_annualized_return = (1 + portfolio_returns.mean()) ** 252 - 1
sti_annualized_return = (1 + sti_returns.mean()) ** 252 - 1
annualized_alpha = portfolio_annualized_return - risk_free_rate - beta * (sti_annualized_return - risk_free_rate)

# Sharpe Ratio
sharpe_ratio = (portfolio_returns.mean() * 252 - risk_free_rate) / portfolio_std_dev

# Annualized Alpha: alpha = (portfolio return - risk-free rate) - beta * (market return - risk-free rate)
portfolio_annualized_return = (1 + portfolio_returns.mean()) ** 252 - 1
sti_annualized_return = (1 + sti_returns.mean()) ** 252 - 1
annualized_alpha = portfolio_annualized_return - risk_free_rate - beta * (sti_annualized_return - risk_free_rate)

# Max Drawdown
cumulative_returns = (1 + portfolio_returns).cumprod()
peak = cumulative_returns.cummax()
drawdown = (cumulative_returns - peak) / peak
max_drawdown = drawdown.min()  # Max drawdown is the lowest value of the drawdown

# Value at Risk (VaR)
var_5th = np.percentile(portfolio_returns, 5)  # 5th percentile for VaR (5%)
var_11th = np.percentile(portfolio_returns, 11)  # 11th percentile for VaR (11%)

# Print results
print("Total Portfolio Return for 2023:", total_return)
print("Annualized Portfolio Standard Deviation:", portfolio_std_dev)
print("Sharpe Ratio:", sharpe_ratio)
print("Beta:", beta)
print("Annualized Alpha:", annualized_alpha)
print("Max Drawdown (%):", max_drawdown * 100)
print("Value At Risk (5th Percentile):", var_5th)
print("Value At Risk (11th Percentile):", var_11th)

/usr/local/lib/python3.10/dist-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(
[*********************100%***********************]  1 of 1 completed


Optimized Weights with Variant 1: OrderedDict([('BN4.SI', 0.0), ('C38U.SI', 0.0), ('D05.SI', 0.0), ('F34.SI', 0.0), ('S68.SI', 0.3), ('V03.SI', 0.7), ('Z74.SI', 0.0)])
Expected annual return: 7.7%
Annual volatility: 12.9%
Sharpe Ratio: 0.44
Total Portfolio Return for 2023: -0.07319131289413328
Annualized Portfolio Standard Deviation: 0.15358239857305833
Sharpe Ratio: -0.6216426898245692
Beta: 0.7820416822403626
Annualized Alpha: Ticker
^STI   -0.072448
dtype: float64
Max Drawdown (%): -25.554981538063405
Value At Risk (5th Percentile): -0.015166249706071657
Value At Risk (11th Percentile): -0.01106774484657587


## Variant 2 Portfolio : Defensive Allocation

In [73]:
sector_lower = {"Consumer Staples": 0.2, "Real Estate": 0.2}  # At least 20% each
sector_upper = {
    "Consumer Staples": 0.35,  # Max 35% in Consumer Staples
    "Real Estate": 0.35,  # Max 35% in Real Estate
    "Industrials": 0.15,  # Max 15% in Industrials
    "Financials": 0.4  # Max 40% in Financials
}

ef = EfficientFrontier(mu, S)
ef.add_objective(objective_functions.L2_reg, gamma=0.1)
ef.add_sector_constraints(sector_mapper, sector_lower, sector_upper)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

print("Optimized Weights with Variant 1:", cleaned_weights)

ef.portfolio_performance(verbose=True)

w = np.array(list(cleaned_weights.values())) # Convert odict_values to a NumPy array
if w.sum() != 1:
  w = w / w.sum()

daily_returns = newData.pct_change().dropna()
portfolio_returns = daily_returns.dot(w)

total_return = (1 + portfolio_returns).prod() - 1
portfolio_std_dev = portfolio_returns.std() * np.sqrt(252)
risk_free_rate = 0.03

# Download STI data (used as market benchmark)
sti_data = yf.download('^STI', start=start_date, end=end_date)['Adj Close']
sti_returns = sti_data.pct_change().dropna()

# Ensure both portfolio_returns and sti_returns have the same length and are aligned
aligned_returns = pd.concat([portfolio_returns, sti_returns], axis=1).dropna()
portfolio_returns_aligned = aligned_returns.iloc[:, 0]
sti_returns_aligned = aligned_returns.iloc[:, 1]

# Beta calculation: covariance of portfolio with market (STI), divided by variance of market (STI)
cov_matrix = np.cov(portfolio_returns_aligned, sti_returns_aligned)
beta = cov_matrix[0, 1] / cov_matrix[1, 1]

# Annualized Alpha: alpha = (portfolio return - risk-free rate) - beta * (market return - risk-free rate)
portfolio_annualized_return = (1 + portfolio_returns.mean()) ** 252 - 1
sti_annualized_return = (1 + sti_returns.mean()) ** 252 - 1
annualized_alpha = portfolio_annualized_return - risk_free_rate - beta * (sti_annualized_return - risk_free_rate)

# Sharpe Ratio
sharpe_ratio = (portfolio_returns.mean() * 252 - risk_free_rate) / portfolio_std_dev

# Annualized Alpha: alpha = (portfolio return - risk-free rate) - beta * (market return - risk-free rate)
portfolio_annualized_return = (1 + portfolio_returns.mean()) ** 252 - 1
sti_annualized_return = (1 + sti_returns.mean()) ** 252 - 1
annualized_alpha = portfolio_annualized_return - risk_free_rate - beta * (sti_annualized_return - risk_free_rate)

# Max Drawdown
cumulative_returns = (1 + portfolio_returns).cumprod()
peak = cumulative_returns.cummax()
drawdown = (cumulative_returns - peak) / peak
max_drawdown = drawdown.min()  # Max drawdown is the lowest value of the drawdown

# Value at Risk (VaR)
var_5th = np.percentile(portfolio_returns, 5)  # 5th percentile for VaR (5%)
var_11th = np.percentile(portfolio_returns, 11)  # 11th percentile for VaR (11%)

# Print results
print("Total Portfolio Return for 2023:", total_return)
print("Annualized Portfolio Standard Deviation:", portfolio_std_dev)
print("Sharpe Ratio:", sharpe_ratio)
print("Beta:", beta)
print("Annualized Alpha:", annualized_alpha)
print("Max Drawdown (%):", max_drawdown * 100)
print("Value At Risk (5th Percentile):", var_5th)
print("Value At Risk (11th Percentile):", var_11th)

/usr/local/lib/python3.10/dist-packages/pypfopt/efficient_frontier/efficient_frontier.py:259: UserWarning: max_sharpe transforms the optimization problem so additional objectives may not work as expected.
  warnings.warn(
[*********************100%***********************]  1 of 1 completed


Optimized Weights with Variant 1: OrderedDict([('BN4.SI', 0.0), ('C38U.SI', 0.22887), ('D05.SI', 0.0), ('F34.SI', 0.0), ('S68.SI', 0.4), ('V03.SI', 0.35), ('Z74.SI', 0.02113)])
Expected annual return: 4.6%
Annual volatility: 12.1%
Sharpe Ratio: 0.22
Total Portfolio Return for 2023: 0.01921280625284494
Annualized Portfolio Standard Deviation: 0.1185395852700744
Sharpe Ratio: -0.030915532413276076
Beta: 0.821720459358844
Annualized Alpha: Ticker
^STI    0.018682
dtype: float64
Max Drawdown (%): -15.936543902086726
Value At Risk (5th Percentile): -0.012423535398747987
Value At Risk (11th Percentile): -0.0083171140502458


## Benchmark Portfolio

In [71]:
start_date = "2023-01-01"
end_date = "2023-12-31"

stiData = yf.download(["^STI"], start = start_date, end = end_date)

price_data = stiData['Adj Close']

price_data.head()

[*********************100%***********************]  1 of 1 completed


Ticker,^STI
Date,
2023-01-03 00:00:00+00:00,3245.800049
2023-01-04 00:00:00+00:00,3242.459961
2023-01-05 00:00:00+00:00,3292.659912
2023-01-06 00:00:00+00:00,3276.719971
2023-01-09 00:00:00+00:00,3305.669922


In [72]:
daily_returns = price_data.pct_change().dropna()

total_return = (1 + daily_returns).prod() - 1
portfolio_std_dev = daily_returns.std() * np.sqrt(252)
risk_free_rate = 0.03

sharpe_ratio = (daily_returns.mean() * 252 - risk_free_rate) / portfolio_std_dev

print("Total Portfolio Return for 2023:", total_return)
print("Annualized Portfolio Standard Deviation:", portfolio_std_dev)
print("Sharpe Ratio:", sharpe_ratio)

Total Portfolio Return for 2023: Ticker
^STI   -0.001704
dtype: float64
Annualized Portfolio Standard Deviation: Ticker
^STI    0.099808
dtype: float64
Sharpe Ratio: Ticker
^STI   -0.268273
dtype: float64
